# Bernstein-Vazirani Algorithm: Finding Hidden Bit Strings

## Introduction

The Bernstein-Vazirani algorithm, developed by Ethan Bernstein and Umesh Vazirani in 1992, demonstrates how quantum computers can extract specific information from functions more efficiently than classical computers. While similar in structure to the Deutsch-Jozsa algorithm, Bernstein-Vazirani solves a different and arguably more practical problem: finding a hidden bit string.

This tutorial builds on concepts from the Deutsch and Deutsch-Jozsa notebooks, particularly quantum parallelism and phase kickback.

## The Problem

We are given a black-box function (oracle) that computes:

$$f(x) = s \cdot x \pmod{2}$$

where:
- $x$ is an $n$-bit input string: $x = x_0 x_1 \cdots x_{n-1}$
- $s$ is an unknown $n$-bit **secret string**: $s = s_0 s_1 \cdots s_{n-1}$
- $s \cdot x$ is the bitwise inner product (dot product): $s \cdot x = s_0 x_0 \oplus s_1 x_1 \oplus \cdots \oplus s_{n-1} x_{n-1}$
- $\oplus$ represents XOR (addition modulo 2)

**Goal**: Find the hidden bit string $s$.

### Example

If $s = 101$ (in binary), then:
- $f(000) = 1\cdot0 \oplus 0\cdot0 \oplus 1\cdot0 = 0$
- $f(001) = 1\cdot0 \oplus 0\cdot0 \oplus 1\cdot1 = 1$
- $f(010) = 1\cdot0 \oplus 0\cdot1 \oplus 1\cdot0 = 0$
- $f(100) = 1\cdot1 \oplus 0\cdot0 \oplus 1\cdot0 = 1$
- $f(101) = 1\cdot1 \oplus 0\cdot0 \oplus 1\cdot1 = 0$
- etc.

### Classical Solution

To find all $n$ bits of $s$, a classical algorithm must make **$n$ queries**:
1. Query $f(100...0)$ to find $s_0$
2. Query $f(010...0)$ to find $s_1$
3. Query $f(001...0)$ to find $s_2$
4. Continue for all $n$ bits

Each query reveals exactly one bit of the secret string.

### Quantum Solution

The Bernstein-Vazirani algorithm finds the entire secret string with **exactly 1 query** to the oracle!

This represents a **linear speedup**: $O(n)$ classical vs $O(1)$ quantum.

## Comparison with Previous Algorithms

| Algorithm | Problem | Information Extracted | Classical | Quantum | Speedup Type |
|-----------|---------|----------------------|-----------|---------|-------------|
| Deutsch | 1-bit constant/balanced | Global property | 2 queries | 1 query | Constant (2x) |
| Deutsch-Jozsa | n-bit constant/balanced | Global property | $2^{n-1}+1$ | 1 query | Exponential |
| **Bernstein-Vazirani** | **Hidden bit string** | **Specific information** | **n queries** | **1 query** | **Linear** |

While the speedup is "only" linear, the Bernstein-Vazirani algorithm is significant because:
1. It extracts **specific information** (the string $s$) rather than just a global property
2. It demonstrates that quantum algorithms can be deterministic and efficient for information retrieval
3. It's a building block for understanding more complex algorithms like Simon's and Shor's

## The Quantum Circuit

The circuit is nearly identical to Deutsch-Jozsa, but the oracle and interpretation are different:

```
|0⟩ ─── H ─── ┐
|0⟩ ─── H ─── │
|0⟩ ─── H ─── ┤  Uf  ├─── H ─── H ─── H ─── Measure → s₀
  ⋮      ⋮    │      │    ⋮     ⋮     ⋮              ⋮
|0⟩ ─── H ─── ┤      ├─── H ─── H ─── H ─── Measure → sₙ₋₁
|1⟩ ─── H ─── ┘      └
```

### Algorithm Steps:

1. **Initialize**: $n$ input qubits to $|0\rangle$, 1 output qubit to $|1\rangle$
2. **Apply Hadamard to all qubits**: Create superposition
3. **Apply oracle $U_f$**: Encodes the secret string as phases
4. **Apply Hadamard to input qubits**: Extract the secret string
5. **Measure**: The measurement directly gives the secret string $s$!

### Why It Works

The key insight is that the oracle implements:
$$U_f |x\rangle|y\rangle = |x\rangle|y \oplus (s \cdot x)\rangle$$

When applied to the state after initial Hadamards, phase kickback encodes $s$ in the relative phases of the superposition. The final Hadamard transforms these phases back into the computational basis, directly revealing $s$.

In [ ]:
# Import necessary libraries
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_bloch_multivector
import matplotlib.pyplot as plt
import numpy as np

print("Qiskit imported successfully!")

## Oracle Implementation

The oracle for Bernstein-Vazirani implements $f(x) = s \cdot x \pmod{2}$.

### Circuit Construction

For each bit $s_i = 1$ in the secret string:
- Apply a CNOT gate from qubit $i$ to the output qubit

This implements the XOR of all positions where $s_i = 1$, which is exactly $s \cdot x$.

### Example
For $s = 101$:
- Apply CNOT from qubit 0 to output (because $s_0 = 1$)
- Skip qubit 1 (because $s_1 = 0$)
- Apply CNOT from qubit 2 to output (because $s_2 = 1$)

In [ ]:
def bv_oracle(secret_string):
    """
    Creates the Bernstein-Vazirani oracle for a given secret string.
    
    The oracle implements f(x) = s·x (mod 2), where s is the secret string.
    
    Args:
        secret_string: String of '0's and '1's representing the secret (e.g., "101")
    
    Returns:
        QuantumCircuit: Oracle circuit implementing the inner product
    """
    n = len(secret_string)
    oracle = QuantumCircuit(n + 1, name=f's={secret_string}')
    
    # Apply CNOT from qubit i to output qubit for each bit where s_i = 1
    for i, bit in enumerate(reversed(secret_string)):  # reversed because qubit 0 is rightmost
        if bit == '1':
            oracle.cx(i, n)
    
    return oracle


def int_to_binary_string(number, n_bits):
    """
    Converts an integer to a binary string of specified length.
    
    Args:
        number: Integer to convert
        n_bits: Number of bits in output string
    
    Returns:
        str: Binary representation (e.g., 5 with n_bits=4 returns "0101")
    """
    return format(number, f'0{n_bits}b')

## Building the Bernstein-Vazirani Circuit

In [ ]:
def bernstein_vazirani(oracle, n):
    """
    Implements the Bernstein-Vazirani algorithm.
    
    Args:
        oracle: Quantum circuit implementing f(x) = s·x (mod 2)
        n: Number of bits in the secret string
    
    Returns:
        QuantumCircuit: Complete Bernstein-Vazirani circuit
    """
    # Create circuit with n input qubits, 1 output qubit, n classical bits
    qr = QuantumRegister(n + 1, 'q')
    cr = ClassicalRegister(n, 'c')
    bv_circuit = QuantumCircuit(qr, cr)
    
    # Step 1: Initialize output qubit to |1⟩
    bv_circuit.x(n)
    bv_circuit.barrier()
    
    # Step 2: Apply Hadamard gates to all qubits
    for i in range(n + 1):
        bv_circuit.h(i)
    bv_circuit.barrier()
    
    # Step 3: Apply the oracle
    bv_circuit.compose(oracle, inplace=True)
    bv_circuit.barrier()
    
    # Step 4: Apply Hadamard gates to input qubits
    for i in range(n):
        bv_circuit.h(i)
    bv_circuit.barrier()
    
    # Step 5: Measure input qubits
    for i in range(n):
        bv_circuit.measure(i, i)
    
    return bv_circuit

## Example 1: Small Case (n=3)

Let's start with a 3-bit secret string.

In [ ]:
# Test with a 3-bit secret string
secret = "101"
n = len(secret)

print(f"Secret string: {secret}")
print(f"Classical approach: {n} queries needed")
print(f"Quantum approach: 1 query\n")
print("=" * 60)

# Create and visualize the oracle
oracle = bv_oracle(secret)
print("\nOracle Circuit:")
display(oracle.draw('mpl'))

# Create the full Bernstein-Vazirani circuit
bv_circuit = bernstein_vazirani(oracle, n)
print("\nComplete Bernstein-Vazirani Circuit:")
display(bv_circuit.draw('mpl', fold=-1))

In [ ]:
# Run the circuit
simulator = AerSimulator()
job = simulator.run(bv_circuit, shots=1000)
result = job.result()
counts = result.get_counts()

print(f"Secret string: {secret}")
print(f"Measurement results: {counts}")
print(f"\nRecovered string: {max(counts, key=counts.get)}")
print(f"Success rate: {max(counts.values()) / 1000 * 100:.1f}%")

# Visualize results
plot_histogram(counts)
plt.title(f'Bernstein-Vazirani Results (secret = {secret})')
plt.show()

## Example 2: Testing Multiple Secret Strings (n=4)

Let's test several different 4-bit secret strings to verify the algorithm works for any $s$.

In [ ]:
n = 4
test_secrets = ["0000", "1111", "1010", "0101", "1100", "1001"]

print(f"Testing Bernstein-Vazirani with n={n} qubits")
print(f"Classical: {n} queries | Quantum: 1 query\n")
print("=" * 60)

results = {}

for secret in test_secrets:
    oracle = bv_oracle(secret)
    circuit = bernstein_vazirani(oracle, n)
    
    job = simulator.run(circuit, shots=1000)
    result = job.result()
    counts = result.get_counts()
    results[secret] = counts
    
    recovered = max(counts, key=counts.get)
    success_rate = max(counts.values()) / 1000 * 100
    
    print(f"Secret: {secret} | Recovered: {recovered} | Success: {success_rate:.1f}%")

In [ ]:
# Visualize results for all test cases
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, (secret, counts) in enumerate(results.items()):
    plot_histogram(counts, ax=axes[idx])
    axes[idx].set_title(f'Secret string: {secret}')

plt.tight_layout()
plt.show()

## Example 3: Scaling Up (n=8)

With 8 bits, the classical algorithm needs 8 queries, while the quantum algorithm still needs only 1.

In [ ]:
n = 8
secret = "10110101"

print(f"Testing with {n}-bit secret string")
print(f"Secret: {secret}")
print(f"Classical: {n} queries | Quantum: 1 query")
print(f"Speedup: {n}x\n")
print("=" * 60)

oracle = bv_oracle(secret)
circuit = bernstein_vazirani(oracle, n)

print(f"\nCircuit depth: {circuit.depth()}")
print(f"Circuit size: {circuit.size()} gates")

job = simulator.run(circuit, shots=1000)
result = job.result()
counts = result.get_counts()

recovered = max(counts, key=counts.get)
success_rate = max(counts.values()) / 1000 * 100

print(f"\nRecovered string: {recovered}")
print(f"Success rate: {success_rate:.1f}%")
print(f"Match: {recovered == secret}")

## Demonstrating Linear Speedup

Let's visualize how the classical query complexity grows linearly with $n$, while quantum stays constant.

In [ ]:
# Compare query complexity
n_values = range(1, 21)
classical_queries = list(n_values)
quantum_queries = [1] * len(n_values)

plt.figure(figsize=(12, 6))
plt.plot(n_values, classical_queries, 'ro-', label='Classical', linewidth=2, markersize=8)
plt.plot(n_values, quantum_queries, 'bs-', label='Quantum (Bernstein-Vazirani)', linewidth=2, markersize=8)
plt.xlabel('Number of bits (n)', fontsize=12)
plt.ylabel('Number of oracle queries', fontsize=12)
plt.title('Bernstein-Vazirani Algorithm: Linear Speedup', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11, loc='upper left')
plt.xlim(0, 21)
plt.ylim(0, 22)

# Add annotations
for n in [5, 10, 15, 20]:
    plt.annotate(f'{n}x', 
                xy=(n, n), xytext=(n+1, n+2),
                arrowprops=dict(arrowstyle='->', color='red', alpha=0.7),
                fontsize=10, color='red')

plt.tight_layout()
plt.show()

print("\nQuery Complexity Comparison:")
print("=" * 50)
print(f"{'n':<5} {'Classical':<15} {'Quantum':<10} {'Speedup'}")
print("=" * 50)
for n in [1, 2, 4, 8, 16, 32, 64, 128]:
    classical = n
    quantum = 1
    speedup = classical / quantum
    print(f"{n:<5} {classical:<15} {quantum:<10} {speedup:.0f}x")

## Mathematical Walkthrough

Let's trace through the quantum state evolution for a concrete example: $s = 101$ (3 bits).

### Step 1: Initial State
$$|\psi_0\rangle = |000\rangle|1\rangle$$

### Step 2: After Hadamards
$$|\psi_1\rangle = H^{\otimes 3}|000\rangle \otimes H|1\rangle$$
$$= \frac{1}{2\sqrt{2}} \sum_{x=0}^{7} |x\rangle \otimes (|0\rangle - |1\rangle)$$
$$= \frac{1}{2\sqrt{2}} \sum_{x=0}^{7} |x\rangle \otimes |-\rangle$$

This is a uniform superposition over all possible 3-bit inputs.

### Step 3: After Oracle (Phase Kickback)

The oracle applies $|x\rangle|-\rangle \rightarrow (-1)^{s \cdot x}|x\rangle|-\rangle$

For $s = 101$:
$$|\psi_2\rangle = \frac{1}{2\sqrt{2}} \sum_{x=0}^{7} (-1)^{s \cdot x} |x\rangle \otimes |-\rangle$$

Let's compute $s \cdot x$ for each $x$:
- $x=000$: $s \cdot x = 0$ → phase $+1$
- $x=001$: $s \cdot x = 1$ → phase $-1$
- $x=010$: $s \cdot x = 0$ → phase $+1$
- $x=011$: $s \cdot x = 1$ → phase $-1$
- $x=100$: $s \cdot x = 1$ → phase $-1$
- $x=101$: $s \cdot x = 0$ → phase $+1$
- $x=110$: $s \cdot x = 1$ → phase $-1$
- $x=111$: $s \cdot x = 0$ → phase $+1$

### Step 4: After Final Hadamards

The Hadamard transform on the input qubits:
$$H^{\otimes n}|x\rangle = \frac{1}{2^{n/2}} \sum_{z=0}^{2^n-1} (-1)^{x \cdot z} |z\rangle$$

Applying this:
$$|\psi_3\rangle = \frac{1}{2^n} \sum_{x=0}^{2^n-1} \sum_{z=0}^{2^n-1} (-1)^{s \cdot x + x \cdot z} |z\rangle$$

### Key Mathematical Insight

The amplitude of state $|z\rangle$ is:
$$A_z = \frac{1}{2^n} \sum_{x=0}^{2^n-1} (-1)^{x \cdot (s \oplus z)}$$

This sum equals:
- $1$ if $z = s$ (all terms have the same phase)
- $0$ if $z \neq s$ (equal positive and negative terms cancel)

Therefore: $|\psi_3\rangle = |s\rangle$

**The measurement deterministically yields the secret string $s$!**

## Verifying Classical Queries

Let's demonstrate why classical algorithms need $n$ queries by simulating the classical approach.

In [ ]:
def classical_bv(secret):
    """
    Simulates the classical approach to finding the secret string.
    
    Args:
        secret: The hidden bit string
    
    Returns:
        tuple: (recovered_secret, number_of_queries)
    """
    n = len(secret)
    recovered = ['?'] * n
    queries = 0
    
    print(f"Finding secret string of length {n}...\n")
    
    # Query each basis vector to find each bit of s
    for i in range(n):
        # Create query with single 1 at position i
        query = ['0'] * n
        query[i] = '1'
        query_string = ''.join(reversed(query))  # reverse for bit ordering
        
        # Compute f(query) = s · query
        result = int(secret[i]) ^ 0  # Only bit i contributes
        result = int(secret[n-1-i])  # Adjust for bit ordering
        
        queries += 1
        recovered[n-1-i] = str(result)
        
        print(f"Query {queries}: f({query_string}) = {result}")
        print(f"  → Discovered: s[{i}] = {result}")
        print(f"  → Current progress: {''.join(recovered)}\n")
    
    final_secret = ''.join(recovered)
    return final_secret, queries

# Test classical approach
secret = "1011"
print(f"Actual secret: {secret}")
print("=" * 60)
recovered, num_queries = classical_bv(secret)
print("=" * 60)
print(f"Recovered: {recovered}")
print(f"Total queries: {num_queries}")
print(f"\nQuantum approach would need: 1 query")
print(f"Speedup: {num_queries}x")

## Key Insights

### 1. Quantum Parallelism with Information Extraction

Unlike Deutsch-Jozsa (which extracts a global property), Bernstein-Vazirani extracts **specific information** - the entire secret string. The algorithm queries the function once on a superposition of all $2^n$ inputs, and the interference pattern directly reveals $s$.

### 2. The Power of Phase Encoding

The oracle encodes the secret string into the **phases** of the quantum state:
$$|\psi\rangle = \sum_x (-1)^{s \cdot x} |x\rangle$$

The final Hadamard transforms these phases back into computational basis states, concentrating all amplitude on $|s\rangle$.

### 3. Deterministic Algorithm

Like Deutsch-Jozsa, this algorithm is **deterministic**:
- Always recovers the correct secret string
- With 100% probability
- In a single query

### 4. Comparison to Classical Information Theory

Classically, to learn $n$ bits of information requires at least $n$ binary queries. The Bernstein-Vazirani algorithm appears to violate this by learning $n$ bits from 1 query. However:
- The quantum query accesses the function on **all** $2^n$ inputs simultaneously
- The oracle performs $2^n$ operations in superposition
- This demonstrates quantum parallelism's power for information extraction

### 5. Building Block for Advanced Algorithms

The techniques used here are fundamental to:
- **Simon's Algorithm**: Period finding with exponential speedup
- **Quantum Fourier Transform**: Phase extraction and transformation
- **Shor's Algorithm**: Factoring integers exponentially faster

## Recursive Bernstein-Vazirani (Advanced)

An interesting extension: what if we want to find a secret string where we can only query a sub-function at a time? Let's implement a simple demonstration.

In [ ]:
# Test multiple random secret strings
n = 6
num_tests = 5

print(f"Testing {num_tests} random {n}-bit secret strings\n")
print("=" * 60)

for test in range(num_tests):
    # Generate random secret string
    secret_int = np.random.randint(0, 2**n)
    secret = int_to_binary_string(secret_int, n)
    
    # Run Bernstein-Vazirani
    oracle = bv_oracle(secret)
    circuit = bernstein_vazirani(oracle, n)
    
    job = simulator.run(circuit, shots=1)
    result = job.result()
    counts = result.get_counts()
    recovered = list(counts.keys())[0]
    
    match = "✓" if recovered == secret else "✗"
    print(f"Test {test+1}: Secret={secret} | Recovered={recovered} | {match}")

print("=" * 60)
print(f"\nAll tests successful! Each required only 1 quantum query.")
print(f"Classical approach would have needed {n} queries per test.")
print(f"Total quantum queries: {num_tests}")
print(f"Total classical queries would be: {num_tests * n}")
print(f"Overall speedup: {num_tests * n / num_tests}x")

## Practical Considerations

### Advantages
1. **Deterministic**: Always gives the correct answer
2. **Scalable**: Works for any size $n$
3. **Simple**: Uses only Hadamard gates, CNOT gates, and measurements
4. **Educational**: Clearly demonstrates quantum parallelism

### Limitations
1. **Specific Problem**: Only solves the inner product problem
2. **Oracle Assumption**: Requires access to a quantum oracle for $f(x) = s \cdot x$
3. **Noise Sensitivity**: On real quantum hardware, noise can corrupt the result
4. **Not Directly Practical**: The specific problem isn't commonly encountered

### Historical Significance
- Demonstrated that quantum algorithms can extract specific information efficiently
- Introduced phase-based encoding techniques used in later algorithms
- Helped inspire Simon's algorithm, which led to Shor's factoring algorithm
- One of the clearest demonstrations of quantum parallelism

## Comparison Summary

### Query Complexity
| Algorithm | Classical | Quantum | Type |
|-----------|-----------|---------|------|
| Deutsch | 2 | 1 | Constant |
| Deutsch-Jozsa | $2^{n-1}+1$ | 1 | Exponential |
| Bernstein-Vazirani | $n$ | 1 | Linear |
| Simon (next) | $O(n)$ | $O(n)$ but exponentially better* | Exponential |
| Grover (future) | $O(N)$ | $O(\sqrt{N})$ | Quadratic |

*Simon's algorithm requires $O(n)$ queries but finds information that would take exponential time classically

### Circuit Complexity
- Gates: $O(n)$ (linear in problem size)
- Depth: $O(1)$ for oracle + $O(1)$ for Hadamards = $O(1)$
- Qubits: $n+1$

## Conclusion

The Bernstein-Vazirani algorithm elegantly demonstrates:

1. **Quantum Information Extraction**: Quantum algorithms can extract $n$ bits of specific information with a single query through quantum parallelism

2. **Phase-Based Encoding**: The secret string is encoded in quantum phases via $(-1)^{s \cdot x}$, which are then decoded by Hadamard transforms

3. **Linear Speedup**: While not exponential like Deutsch-Jozsa, the $n$-to-1 reduction is still significant and demonstrates a different type of quantum advantage

4. **Foundation for Advanced Algorithms**: The techniques of phase encoding and Hadamard-based interference are crucial building blocks for Simon's algorithm, quantum Fourier transform, and ultimately Shor's factoring algorithm

5. **Deterministic Quantum Computing**: Shows that quantum algorithms can be deterministic and provide exact answers, not just probabilistic improvements

### Next Steps

Building on Bernstein-Vazirani, you can explore:
- **Simon's Algorithm**: Uses similar principles to find hidden periods with exponential speedup
- **Quantum Fourier Transform**: Generalizes the Hadamard transform for phase manipulation
- **Shor's Algorithm**: Applies these techniques to factor integers exponentially faster
- **Grover's Algorithm**: Different approach using amplitude amplification for search

The journey from Deutsch → Deutsch-Jozsa → Bernstein-Vazirani → Simon → Shor shows how quantum computing concepts build upon each other to achieve remarkable computational advantages!